## 1.语言模型(n-gram) 

假设一个词的出现只与前面n个词相关。
缺陷：参数空间过大、数据稀疏

In [1]:
#读取数据
with open('jay.txt',encoding="utf-8") as f:
    corpus_chars = f.read()
print(len(corpus_chars))
print(corpus_chars[: 40])
corpus_chars = corpus_chars.replace('\n', ' ').replace('\r', ' ')
corpus_chars = corpus_chars[: 10000]

585
想要有直升机
想要和你飞到宇宙去
想要和你融化在一起
融化在宇宙里
我每天每天每


In [2]:
# 建立字符索引
#去重，得到索引到字符映射
idx_to_char=list(set(corpus_chars))
#字符到索引的映射
char_to_idx={char:i for i,char in enumerate(idx_to_char)}
vocab_size=len(char_to_idx)
print(vocab_size)
#将每个字符转换为索引，得到索引序列
corpus_indices=[char_to_idx[char] for char in corpus_chars]
sample=corpus_indices[:20]
print('chars:', ''.join([idx_to_char[idx] for idx in sample]))
print('indices:', sample)

74
chars: 想要有直升机 想要和你飞到宇宙去 想要和
indices: [19, 16, 36, 5, 26, 39, 52, 19, 16, 13, 62, 2, 51, 29, 35, 44, 52, 19, 16, 13]


In [3]:
def load_data_jay_lyrics():
    with open('jay.txt',encoding="utf-8") as f:
        corpus_chars = f.read()
    corpus_chars = corpus_chars.replace('\n', ' ').replace('\r', ' ')
    corpus_chars = corpus_chars[0:10000]
    idx_to_char = list(set(corpus_chars))
    char_to_idx = dict([(char, i) for i, char in enumerate(idx_to_char)])
    vocab_size = len(char_to_idx)
    corpus_indices = [char_to_idx[char] for char in corpus_chars]
    return corpus_indices, char_to_idx, idx_to_char, vocab_size

### 时序数据采样

时序数据中通常包含连续字符，选定时间步数n对长度为T的序列采样，可以得到T-n个合法样本，但样本会存在大量重合

(1)随机采样
每次在数据里随机采样一个小批量

In [4]:
import torch
import random
def data_iter_random(corpus_indices, batch_size, num_steps, device=None):
    # 减1是因为对于长度为n的序列，X最多只有包含其中的前n - 1个字符
    num_examples = (len(corpus_indices) - 1) // num_steps  # 下取整，得到不重叠情况下的样本个数
    example_indices = [i * num_steps for i in range(num_examples)]  # 每个样本的第一个字符在corpus_indices中的下标
    random.shuffle(example_indices)

    def _data(i):
        # 返回从i开始的长为num_steps的序列
        return corpus_indices[i: i + num_steps]
    if device is None:
        device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    
    for i in range(0, num_examples, batch_size):
        # 每次选出batch_size个随机样本
        batch_indices = example_indices[i: i + batch_size]  # 当前batch的各个样本的首字符的下标
        X = [_data(j) for j in batch_indices]
        Y = [_data(j + 1) for j in batch_indices]
        yield torch.tensor(X, device=device), torch.tensor(Y, device=device)

my_seq = list(range(30))
for X, Y in data_iter_random(my_seq, batch_size=2, num_steps=6):
    print('X: ', X, '\nY:', Y, '\n')

X:  tensor([[18, 19, 20, 21, 22, 23],
        [12, 13, 14, 15, 16, 17]]) 
Y: tensor([[19, 20, 21, 22, 23, 24],
        [13, 14, 15, 16, 17, 18]]) 

X:  tensor([[ 6,  7,  8,  9, 10, 11],
        [ 0,  1,  2,  3,  4,  5]]) 
Y: tensor([[ 7,  8,  9, 10, 11, 12],
        [ 1,  2,  3,  4,  5,  6]]) 



 (2)相邻采样
相邻的两个随机小批量在原始序列上的位置相毗邻

In [5]:
def data_iter_consecutive(corpus_indices, batch_size, num_steps, device=None):
    if device is None:
        device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    corpus_len = len(corpus_indices) // batch_size * batch_size  # 保留下来的序列的长度
    corpus_indices = corpus_indices[: corpus_len]  # 仅保留前corpus_len个字符
    indices = torch.tensor(corpus_indices, device=device)
    indices = indices.view(batch_size, -1)  # resize成(batch_size, )
    batch_num = (indices.shape[1] - 1) // num_steps
    for i in range(batch_num):
        i = i * num_steps
        X = indices[:, i: i + num_steps]
        Y = indices[:, i + 1: i + num_steps + 1]
        yield X, Y
for X, Y in data_iter_consecutive(my_seq, batch_size=2, num_steps=6):
    print('X: ', X, '\nY:', Y, '\n')

X:  tensor([[ 0,  1,  2,  3,  4,  5],
        [15, 16, 17, 18, 19, 20]]) 
Y: tensor([[ 1,  2,  3,  4,  5,  6],
        [16, 17, 18, 19, 20, 21]]) 

X:  tensor([[ 6,  7,  8,  9, 10, 11],
        [21, 22, 23, 24, 25, 26]]) 
Y: tensor([[ 7,  8,  9, 10, 11, 12],
        [22, 23, 24, 25, 26, 27]]) 



## 2.RNN基础 

### 2.1从零实现

In [1]:
 # -*- coding: utf-8 -*-
import torch
import torch.nn as nn
import time
import math
import sys
def load_data_jay_lyrics():
    with open('jay.txt',encoding="utf-8") as f:
        corpus_chars = f.read()
    corpus_chars = corpus_chars.replace('\n', ' ').replace('\r', ' ')
    corpus_chars = corpus_chars[0:10000]
    idx_to_char = list(set(corpus_chars))
    char_to_idx = dict([(char, i) for i, char in enumerate(idx_to_char)])
    vocab_size = len(char_to_idx)
    corpus_indices = [char_to_idx[char] for char in corpus_chars]
    return corpus_indices, char_to_idx, idx_to_char, vocab_size
(corpus_indices, char_to_idx, idx_to_char, vocab_size) = load_data_jay_lyrics()
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [2]:
#one-hot 将词典向量转换为one-hot
def one_hot(x, n_class, dtype=torch.float32):
    result = torch.zeros(x.shape[0], n_class, dtype=dtype, device=x.device)  # shape: (n, n_class)
    result.scatter_(1, x.long().view(-1, 1), 1)  # result[i, x[i, 0]] = 1
    return result
    
x = torch.tensor([0, 2])
x_one_hot = one_hot(x, vocab_size)
print(x_one_hot)
print(x_one_hot.shape)
print(x_one_hot.sum(axis=1))

tensor([[1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0.],
        [0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0.]])
torch.Size([2, 76])
tensor([1., 1.])


In [3]:
def to_onehot(X, n_class):
    return [one_hot(X[:, i], n_class) for i in range(X.shape[1])]

X = torch.arange(10).view(2, 5)
inputs = to_onehot(X, vocab_size)
print(len(inputs), inputs[0].shape)

5 torch.Size([2, 76])


In [4]:
#参数初始化
num_inputs, num_hiddens, num_outputs = vocab_size, 256, vocab_size
# num_inputs: d
# num_hiddens: h, 隐藏单元的个数是超参数
# num_outputs: q

def get_params():
    def _one(shape):
        param = torch.zeros(shape, device=device, dtype=torch.float32)
        nn.init.normal_(param, 0, 0.01)
        return torch.nn.Parameter(param)

    # 隐藏层参数
    W_xh = _one((num_inputs, num_hiddens))
    W_hh = _one((num_hiddens, num_hiddens))
    b_h = torch.nn.Parameter(torch.zeros(num_hiddens, device=device))
    # 输出层参数
    W_hq = _one((num_hiddens, num_outputs))
    b_q = torch.nn.Parameter(torch.zeros(num_outputs, device=device))
    return (W_xh, W_hh, b_h, W_hq, b_q)

In [5]:
def rnn(inputs, state, params):
    # inputs和outputs皆为num_steps个形状为(batch_size, vocab_size)的矩阵
    W_xh, W_hh, b_h, W_hq, b_q = params
    H, = state
    outputs = []
    for X in inputs:
        H = torch.tanh(torch.matmul(X, W_xh) + torch.matmul(H, W_hh) + b_h)
        Y = torch.matmul(H, W_hq) + b_q
        outputs.append(Y)
    return outputs, (H,)
#初始化隐藏变量
def init_rnn_state(batch_size, num_hiddens, device):
    return (torch.zeros((batch_size, num_hiddens), device=device), )
state = init_rnn_state(X.shape[0], num_hiddens, device)
inputs = to_onehot(X.to(device), vocab_size)
params = get_params()
outputs, state_new = rnn(inputs, state, params)
#梯度裁剪
def grad_clipping(params, theta, device):
    norm = torch.tensor([0.0], device=device)
    for param in params:
        norm += (param.grad.data ** 2).sum()
    norm = norm.sqrt().item()
    if norm > theta:
        for param in params:
            param.grad.data *= (theta / norm)


In [6]:
#定义预测函数，基于前缀prefix预测接下来num_chars个字符
def predict_rnn(prefix, num_chars, rnn, params, init_rnn_state,
                num_hiddens, vocab_size, device, idx_to_char, char_to_idx):
    state = init_rnn_state(1, num_hiddens, device)
    # output记录prefix加上预测的num_chars个字符
    output = [char_to_idx[prefix[0]]]   
    for t in range(num_chars + len(prefix) - 1):
        # 将上一时间步的输出作为当前时间步的输入
        X = to_onehot(torch.tensor([[output[-1]]], device=device), vocab_size)
        # 计算输出和更新隐藏状态
        (Y, state) = rnn(X, state, params)
        # 下一个时间步的输入是prefix里的字符或者当前的最佳预测字符
        if t < len(prefix) - 1:
            output.append(char_to_idx[prefix[t + 1]])
        else:
            output.append(Y[0].argmax(dim=1).item())
    return ''.join([idx_to_char[i] for i in output])
#一个简单的预测实现
predict_rnn('分开', 10, rnn, params, init_rnn_state, num_hiddens, vocab_size,
            device, idx_to_char, char_to_idx)

'分开宙世起尽面温但蜜护飞'

### 困惑度

困惑度可用于评价语言模型的好坏,是对交叉熵损失函数做指数运算后得到的值：
(1)最佳情况下，模型总是把标签类别的概率预测为1，困惑都为1；
(2)最坏抢矿下，总是把标签类别的概率预测为0，困惑都为正无穷；
(3)基线情况下，模型总是预测所有类别的概率都相同，此时困惑都为类别个数

### 模型训练

(1)使用困惑都评价模型；
(2)在迭代模型参数前裁剪梯度；
(3)对时序数据采用不同采样方法将导致状态初始化的不同

In [7]:
#随机采样
import torch
import random
def data_iter_random(corpus_indices, batch_size, num_steps, device=None):
    # 减1是因为对于长度为n的序列，X最多只有包含其中的前n - 1个字符
    num_examples = (len(corpus_indices) - 1) // num_steps  # 下取整，得到不重叠情况下的样本个数
    example_indices = [i * num_steps for i in range(num_examples)]  # 每个样本的第一个字符在corpus_indices中的下标
    random.shuffle(example_indices)

    def _data(i):
        # 返回从i开始的长为num_steps的序列
        return corpus_indices[i: i + num_steps]
    if device is None:
        device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    
    for i in range(0, num_examples, batch_size):
        # 每次选出batch_size个随机样本
        batch_indices = example_indices[i: i + batch_size]  # 当前batch的各个样本的首字符的下标
        X = [_data(j) for j in batch_indices]
        Y = [_data(j + 1) for j in batch_indices]
        yield torch.tensor(X, device=device), torch.tensor(Y, device=device)
#相邻采样
def data_iter_consecutive(corpus_indices, batch_size, num_steps, device=None):
    if device is None:
        device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    corpus_len = len(corpus_indices) // batch_size * batch_size  # 保留下来的序列的长度
    corpus_indices = corpus_indices[: corpus_len]  # 仅保留前corpus_len个字符
    indices = torch.tensor(corpus_indices, device=device)
    indices = indices.view(batch_size, -1)  # resize成(batch_size, )
    batch_num = (indices.shape[1] - 1) // num_steps
    for i in range(batch_num):
        i = i * num_steps
        X = indices[:, i: i + num_steps]
        Y = indices[:, i + 1: i + num_steps + 1]
        yield X, Y
#定义SGD优化函数
def sgd(params,lr,batch_size):
    for param in params:
        param.data-=lr*param.grad/batch_size

In [8]:
#定义模型训练函数
def train_and_predict_rnn(rnn, get_params, init_rnn_state, num_hiddens,
                          vocab_size, device, corpus_indices, idx_to_char,
                          char_to_idx, is_random_iter, num_epochs, num_steps,
                          lr, clipping_theta, batch_size, pred_period,
                          pred_len, prefixes):
    if is_random_iter:
        data_iter_fn = data_iter_random
    else:
        data_iter_fn = data_iter_consecutive
    params = get_params()
    loss = nn.CrossEntropyLoss()

    for epoch in range(num_epochs):
        if not is_random_iter:  # 如使用相邻采样，在epoch开始时初始化隐藏状态
            state = init_rnn_state(batch_size, num_hiddens, device)
        l_sum, n, start = 0.0, 0, time.time()
        data_iter = data_iter_fn(corpus_indices, batch_size, num_steps, device)
        for X, Y in data_iter:
            if is_random_iter:  # 如使用随机采样，在每个小批量更新前初始化隐藏状态
                state = init_rnn_state(batch_size, num_hiddens, device)
            else:  # 否则需要使用detach函数从计算图分离隐藏状态
                for s in state:
                    s.detach_()
            # inputs是num_steps个形状为(batch_size, vocab_size)的矩阵
            inputs = to_onehot(X, vocab_size)
            # outputs有num_steps个形状为(batch_size, vocab_size)的矩阵
            (outputs, state) = rnn(inputs, state, params)
            # 拼接之后形状为(num_steps * batch_size, vocab_size)
            outputs = torch.cat(outputs, dim=0)
            # Y的形状是(batch_size, num_steps)，转置后再变成形状为
            # (num_steps * batch_size,)的向量，这样跟输出的行一一对应
            y = torch.flatten(Y.T)
            # 使用交叉熵损失计算平均分类误差
            l = loss(outputs, y.long())
            
            # 梯度清0
            if params[0].grad is not None:
                for param in params:
                    param.grad.data.zero_()
            l.backward()
            grad_clipping(params, clipping_theta, device)  # 裁剪梯度
            sgd(params, lr, 1)  # 因为误差已经取过均值，梯度不用再做平均
            l_sum += l.item() * y.shape[0]
            n += y.shape[0]

        if (epoch + 1) % pred_period == 0:
            print('epoch %d, perplexity %f, time %.2f sec' % (
                epoch + 1, math.exp(l_sum / n), time.time() - start))
            for prefix in prefixes:
                print(' -', predict_rnn(prefix, pred_len, rnn, params, init_rnn_state,
                    num_hiddens, vocab_size, device, idx_to_char, char_to_idx))

In [9]:
#以分开，不分开开头，创作长度为50的歌词
num_epochs, num_steps, batch_size, lr, clipping_theta = 250, 35, 16, 1e2, 1e-2
pred_period, pred_len, prefixes = 50, 50, ['分开', '不分开']
#使用随机采样方法
train_and_predict_rnn(rnn, get_params, init_rnn_state, num_hiddens,
                      vocab_size, device, corpus_indices, idx_to_char,
                      char_to_idx, True, num_epochs, num_steps, lr,
                      clipping_theta, batch_size, pred_period, pred_len,
                      prefixes)
#使用相邻采样方法
train_and_predict_rnn(rnn, get_params, init_rnn_state, num_hiddens,
                      vocab_size, device, corpus_indices, idx_to_char,
                      char_to_idx, False, num_epochs, num_steps, lr,
                      clipping_theta, batch_size, pred_period, pred_len,
                      prefixes)

epoch 50, perplexity 5.407897, time 0.03 sec
 - 分开 坏坏的让我感动的可爱女人 温柔的让我 动的的让我感动的可爱女人 温柔的让我 动的的让我感动的可爱女
 - 不分开 想坏的让我感动的可爱女人 温柔的让我 动的的让我感动的可爱女人 温柔的让我 动的的让我感动的可爱女
epoch 100, perplexity 3.626470, time 0.03 sec
 - 分开想想想想想想想想想想想想想想想想想想想想想想想想想想想想想想想想想想想想想想想想想想想想想想想想想想
 - 不分开想想想想想想想想想想想想想想想想想想想想想想想想想想想想想想想想想想想想想想想想想想想想想想想想想想
epoch 150, perplexity 1.846472, time 0.03 sec
 - 分开的让我感动的可爱女人 透明的让我感动的可爱女人 温柔的让我心疼的可爱女人 温柔的让我心疼的可爱女人 
 - 不分开想想想想想著你 这样的让我感动的可爱女人 透明的让我感动的可爱女人 温柔的让我心疼的可爱女人 温柔的
epoch 200, perplexity 1.265632, time 0.03 sec
 - 分开的可爱女人 坏坏的让我疯狂的可爱女人 坏坏的让我疯狂的可爱女人 坏坏的让我感动的可爱女人 坏坏的让我
 - 不分开力全力的让我面红你 漂亮的让我面红的可爱女人 坏坏的让我感动的可爱女人 坏坏的让我疯狂的可爱女人 坏
epoch 250, perplexity 1.807301, time 0.04 sec
 - 分开的可爱女人 温柔的让我心疼的可爱女人 透明的让我感动的可爱女人 坏坏的让我疯狂的可爱女人 漂亮的让我
 - 不分开力引力想想想想想著你小小小的蚂蚁 但我漂亮的可爱女人 漂亮的让我面红的可爱女人 透明的让我感动的可爱
epoch 50, perplexity 15.218496, time 0.03 sec
 - 分开 我心的让我心疼的让我心疼的让我心疼的让我心疼的让我心疼的让我心疼的让我心疼的让我心疼的让我心疼的让
 - 不分开 我我心的让我心疼的让我心疼的让我心疼的让我心疼的让我心疼的让我心疼的让我心疼的让我心疼的让我心疼的
epoch 100, perplexity 3.838174, time 0.03 sec
 

### 2.2 简洁实现

In [10]:
#使用pytorch中的nn.RNN构建循环神经网络
rnn_layer = nn.RNN(input_size=vocab_size, hidden_size=num_hiddens)
num_steps, batch_size = 35, 2
#定义RNN模型
class RNNModel(nn.Module):
    def __init__(self, rnn_layer, vocab_size):
        super(RNNModel, self).__init__()
        self.rnn = rnn_layer
        self.hidden_size = rnn_layer.hidden_size * (2 if rnn_layer.bidirectional else 1) 
        self.vocab_size = vocab_size
        self.dense = nn.Linear(self.hidden_size, vocab_size)

    def forward(self, inputs, state):
        # inputs.shape: (batch_size, num_steps)
        X = to_onehot(inputs, vocab_size)
        X = torch.stack(X)  # X.shape: (num_steps, batch_size, vocab_size)
        hiddens, state = self.rnn(X, state)
        hiddens = hiddens.view(-1, hiddens.shape[-1])  # hiddens.shape: (num_steps * batch_size, hidden_size)
        output = self.dense(hiddens)
        return output, state

In [11]:
#定义训练函数
def predict_rnn_pytorch(prefix, num_chars, model, vocab_size, device, idx_to_char,
                      char_to_idx):
    state = None
    output = [char_to_idx[prefix[0]]]  # output记录prefix加上预测的num_chars个字符
    for t in range(num_chars + len(prefix) - 1):
        X = torch.tensor([output[-1]], device=device).view(1, 1)
        (Y, state) = model(X, state)  # 前向计算不需要传入模型参数
        if t < len(prefix) - 1:
            output.append(char_to_idx[prefix[t + 1]])
        else:
            output.append(Y.argmax(dim=1).item())
    return ''.join([idx_to_char[i] for i in output])
model = RNNModel(rnn_layer, vocab_size).to(device)
predict_rnn_pytorch('分开', 10, model, vocab_size, device, idx_to_char, char_to_idx)

'分开碰大大温碰开大碰大在'

In [12]:
#定义相邻采样模型
def train_and_predict_rnn_pytorch(model, num_hiddens, vocab_size, device,
                                corpus_indices, idx_to_char, char_to_idx,
                                num_epochs, num_steps, lr, clipping_theta,
                                batch_size, pred_period, pred_len, prefixes):
    loss = nn.CrossEntropyLoss()
    optimizer = torch.optim.Adam(model.parameters(), lr=lr)
    model.to(device)
    for epoch in range(num_epochs):
        l_sum, n, start = 0.0, 0, time.time()
        data_iter = data_iter_consecutive(corpus_indices, batch_size, num_steps, device) # 相邻采样
        state = None
        for X, Y in data_iter:
            if state is not None:
                # 使用detach函数从计算图分离隐藏状态
                if isinstance (state, tuple): # LSTM, state:(h, c)  
                    state[0].detach_()
                    state[1].detach_()
                else: 
                    state.detach_()
            (output, state) = model(X, state) # output.shape: (num_steps * batch_size, vocab_size)
            y = torch.flatten(Y.T)
            l = loss(output, y.long())
            
            optimizer.zero_grad()
            l.backward()
            grad_clipping(model.parameters(), clipping_theta, device)
            optimizer.step()
            l_sum += l.item() * y.shape[0]
            n += y.shape[0]
        

        if (epoch + 1) % pred_period == 0:
            print('epoch %d, perplexity %f, time %.2f sec' % (
                epoch + 1, math.exp(l_sum / (n+1)), time.time() - start))
            for prefix in prefixes:
                print(' -', predict_rnn_pytorch(
                    prefix, pred_len, model, vocab_size, device, idx_to_char,
                    char_to_idx))

In [13]:
#训练模型
num_epochs, batch_size, lr, clipping_theta = 250, 32, 1e-3, 1e-2
pred_period, pred_len, prefixes = 50, 50, ['分开', '不分开']
train_and_predict_rnn_pytorch(model, num_hiddens, vocab_size, device,
                            corpus_indices, idx_to_char, char_to_idx,
                            num_epochs, num_steps, lr, clipping_theta,
                            batch_size, pred_period, pred_len, prefixes)

epoch 50, perplexity 1.000000, time 0.00 sec
 - 分开碰大大温碰开大碰大在大碰温在大碰大碰大在大碰温在大碰大碰大在大碰温在大碰大碰大在大碰温在大碰大碰大在
 - 不分开碰大大温碰开大碰大在大碰温在大碰大碰大在大碰温在大碰大碰大在大碰温在大碰大碰大在大碰温在大碰大碰大在
epoch 100, perplexity 1.000000, time 0.00 sec
 - 分开碰大大温碰开大碰大在大碰温在大碰大碰大在大碰温在大碰大碰大在大碰温在大碰大碰大在大碰温在大碰大碰大在
 - 不分开碰大大温碰开大碰大在大碰温在大碰大碰大在大碰温在大碰大碰大在大碰温在大碰大碰大在大碰温在大碰大碰大在
epoch 150, perplexity 1.000000, time 0.00 sec
 - 分开碰大大温碰开大碰大在大碰温在大碰大碰大在大碰温在大碰大碰大在大碰温在大碰大碰大在大碰温在大碰大碰大在
 - 不分开碰大大温碰开大碰大在大碰温在大碰大碰大在大碰温在大碰大碰大在大碰温在大碰大碰大在大碰温在大碰大碰大在
epoch 200, perplexity 1.000000, time 0.00 sec
 - 分开碰大大温碰开大碰大在大碰温在大碰大碰大在大碰温在大碰大碰大在大碰温在大碰大碰大在大碰温在大碰大碰大在
 - 不分开碰大大温碰开大碰大在大碰温在大碰大碰大在大碰温在大碰大碰大在大碰温在大碰大碰大在大碰温在大碰大碰大在
epoch 250, perplexity 1.000000, time 0.00 sec
 - 分开碰大大温碰开大碰大在大碰温在大碰大碰大在大碰温在大碰大碰大在大碰温在大碰大碰大在大碰温在大碰大碰大在
 - 不分开碰大大温碰开大碰大在大碰温在大碰大碰大在大碰温在大碰大碰大在大碰温在大碰大碰大在大碰温在大碰大碰大在


## 3.进阶RNN模型 

### 3.1GRU调用 简洁实现 (重置门和更新门)

In [14]:
 # -*- coding: utf-8 -*-
import torch
from torch import nn, optim
import torch.nn.functional as F
import sys
def load_data_jay_lyrics():
    with open('jay.txt',encoding="utf-8") as f:
        corpus_chars = f.read()
    corpus_chars = corpus_chars.replace('\n', ' ').replace('\r', ' ')
    corpus_chars = corpus_chars[0:10000]
    idx_to_char = list(set(corpus_chars))
    char_to_idx = dict([(char, i) for i, char in enumerate(idx_to_char)])
    vocab_size = len(char_to_idx)
    corpus_indices = [char_to_idx[char] for char in corpus_chars]
    return corpus_indices, char_to_idx, idx_to_char, vocab_size
(corpus_indices, char_to_idx, idx_to_char, vocab_size) = load_data_jay_lyrics()
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

#定义RNN模型
class RNNModel(nn.Module):
    def __init__(self, rnn_layer, vocab_size):
        super(RNNModel, self).__init__()
        self.rnn = rnn_layer
        self.hidden_size = rnn_layer.hidden_size * (2 if rnn_layer.bidirectional else 1) 
        self.vocab_size = vocab_size
        self.dense = nn.Linear(self.hidden_size, vocab_size)

    def forward(self, inputs, state):
        # inputs.shape: (batch_size, num_steps)
        X = to_onehot(inputs, vocab_size)
        X = torch.stack(X)  # X.shape: (num_steps, batch_size, vocab_size)
        hiddens, state = self.rnn(X, state)
        hiddens = hiddens.view(-1, hiddens.shape[-1])  # hiddens.shape: (num_steps * batch_size, hidden_size)
        output = self.dense(hiddens)
        return output, state
#定义训练函数
def predict_rnn_pytorch(prefix, num_chars, model, vocab_size, device, idx_to_char,
                      char_to_idx):
    state = None
    output = [char_to_idx[prefix[0]]]  # output记录prefix加上预测的num_chars个字符
    for t in range(num_chars + len(prefix) - 1):
        X = torch.tensor([output[-1]], device=device).view(1, 1)
        (Y, state) = model(X, state)  # 前向计算不需要传入模型参数
        if t < len(prefix) - 1:
            output.append(char_to_idx[prefix[t + 1]])
        else:
            output.append(Y.argmax(dim=1).item())
    return ''.join([idx_to_char[i] for i in output])

In [15]:
num_hiddens=256
num_epochs, num_steps, batch_size, lr, clipping_theta = 160, 35, 32, 1e2, 1e-2
pred_period, pred_len, prefixes = 40, 50, ['分开', '不分开']

lr = 1e-2 # 注意调整学习率
gru_layer = nn.GRU(input_size=vocab_size, hidden_size=num_hiddens)
model = RNNModel(gru_layer, vocab_size).to(device)
train_and_predict_rnn_pytorch(model, num_hiddens, vocab_size, device,
                                corpus_indices, idx_to_char, char_to_idx,
                                num_epochs, num_steps, lr, clipping_theta,
                                batch_size, pred_period, pred_len, prefixes)

epoch 40, perplexity 1.000000, time 0.00 sec
 - 分开信要全全漂可信要要全全漂可信要要全全漂可信要要全全漂可信要要全全漂可信要要全全漂可信要要全全漂可信要
 - 不分开信要要全全漂可信要要全全漂可信要要全全漂可信要要全全漂可信要要全全漂可信要要全全漂可信要要全全漂可信
epoch 80, perplexity 1.000000, time 0.00 sec
 - 分开信要全全漂可信要要全全漂可信要要全全漂可信要要全全漂可信要要全全漂可信要要全全漂可信要要全全漂可信要
 - 不分开信要要全全漂可信要要全全漂可信要要全全漂可信要要全全漂可信要要全全漂可信要要全全漂可信要要全全漂可信
epoch 120, perplexity 1.000000, time 0.00 sec
 - 分开信要全全漂可信要要全全漂可信要要全全漂可信要要全全漂可信要要全全漂可信要要全全漂可信要要全全漂可信要
 - 不分开信要要全全漂可信要要全全漂可信要要全全漂可信要要全全漂可信要要全全漂可信要要全全漂可信要要全全漂可信
epoch 160, perplexity 1.000000, time 0.00 sec
 - 分开信要全全漂可信要要全全漂可信要要全全漂可信要要全全漂可信要要全全漂可信要要全全漂可信要要全全漂可信要
 - 不分开信要要全全漂可信要要全全漂可信要要全全漂可信要要全全漂可信要要全全漂可信要要全全漂可信要要全全漂可信


### 3.2 LSTM简介实现 （重置门，更新门，遗忘门）

In [18]:
num_hiddens=256
num_epochs, num_steps, batch_size, lr, clipping_theta = 160, 35, 32, 1e2, 1e-2
pred_period, pred_len, prefixes = 40, 50, ['分开', '不分开']

lr = 1e-2 # 注意调整学习率
lstm_layer = nn.LSTM(input_size=vocab_size, hidden_size=num_hiddens)
model = RNNModel(lstm_layer, vocab_size)
train_and_predict_rnn_pytorch(model, num_hiddens, vocab_size, device,
                                corpus_indices, idx_to_char, char_to_idx,
                                num_epochs, num_steps, lr, clipping_theta,
                                batch_size, pred_period, pred_len, prefixes)

epoch 40, perplexity 1.000000, time 0.00 sec
 - 分开蜜融蜜融蜜融宇是融蜜融蜜融宇是融蜜融蜜融宇是融蜜融蜜融宇是融蜜融蜜融宇是融蜜融蜜融宇是融蜜融蜜融宇是
 - 不分开蜜是蜜融蜜融蜜融宇是融蜜融蜜融宇是融蜜融蜜融宇是融蜜融蜜融宇是融蜜融蜜融宇是融蜜融蜜融宇是融蜜融蜜融
epoch 80, perplexity 1.000000, time 0.00 sec
 - 分开蜜融蜜融蜜融宇是融蜜融蜜融宇是融蜜融蜜融宇是融蜜融蜜融宇是融蜜融蜜融宇是融蜜融蜜融宇是融蜜融蜜融宇是
 - 不分开蜜是蜜融蜜融蜜融宇是融蜜融蜜融宇是融蜜融蜜融宇是融蜜融蜜融宇是融蜜融蜜融宇是融蜜融蜜融宇是融蜜融蜜融
epoch 120, perplexity 1.000000, time 0.00 sec
 - 分开蜜融蜜融蜜融宇是融蜜融蜜融宇是融蜜融蜜融宇是融蜜融蜜融宇是融蜜融蜜融宇是融蜜融蜜融宇是融蜜融蜜融宇是
 - 不分开蜜是蜜融蜜融蜜融宇是融蜜融蜜融宇是融蜜融蜜融宇是融蜜融蜜融宇是融蜜融蜜融宇是融蜜融蜜融宇是融蜜融蜜融
epoch 160, perplexity 1.000000, time 0.00 sec
 - 分开蜜融蜜融蜜融宇是融蜜融蜜融宇是融蜜融蜜融宇是融蜜融蜜融宇是融蜜融蜜融宇是融蜜融蜜融宇是融蜜融蜜融宇是
 - 不分开蜜是蜜融蜜融蜜融宇是融蜜融蜜融宇是融蜜融蜜融宇是融蜜融蜜融宇是融蜜融蜜融宇是融蜜融蜜融宇是融蜜融蜜融


### 3.3深度循环神经网络 

In [19]:
gru_layer = nn.LSTM(input_size=vocab_size, hidden_size=num_hiddens,num_layers=6)
model = RNNModel(gru_layer, vocab_size).to(device)
train_and_predict_rnn_pytorch(model, num_hiddens, vocab_size, device,
                                corpus_indices, idx_to_char, char_to_idx,
                                num_epochs, num_steps, lr, clipping_theta,
                                batch_size, pred_period, pred_len, prefixes)

epoch 40, perplexity 1.000000, time 0.00 sec
 - 分开想想想想想想想想想想想想想想想想想想想想想想想想想想想想想想想想想想想想想想想想想想想想想想想想想想
 - 不分开想想想想想想想想想想想想想想想想想想想想想想想想想想想想想想想想想想想想想想想想想想想想想想想想想想
epoch 80, perplexity 1.000000, time 0.00 sec
 - 分开想想想想想想想想想想想想想想想想想想想想想想想想想想想想想想想想想想想想想想想想想想想想想想想想想想
 - 不分开想想想想想想想想想想想想想想想想想想想想想想想想想想想想想想想想想想想想想想想想想想想想想想想想想想
epoch 120, perplexity 1.000000, time 0.00 sec
 - 分开想想想想想想想想想想想想想想想想想想想想想想想想想想想想想想想想想想想想想想想想想想想想想想想想想想
 - 不分开想想想想想想想想想想想想想想想想想想想想想想想想想想想想想想想想想想想想想想想想想想想想想想想想想想
epoch 160, perplexity 1.000000, time 0.00 sec
 - 分开想想想想想想想想想想想想想想想想想想想想想想想想想想想想想想想想想想想想想想想想想想想想想想想想想想
 - 不分开想想想想想想想想想想想想想想想想想想想想想想想想想想想想想想想想想想想想想想想想想想想想想想想想想想


### 3.3双向循环神经网络  

In [21]:
num_hiddens=128
num_epochs, num_steps, batch_size, lr, clipping_theta = 160, 35, 32, 1e-2, 1e-2
pred_period, pred_len, prefixes = 40, 50, ['分开', '不分开']

lr = 1e-2 # 注意调整学习率

gru_layer = nn.GRU(input_size=vocab_size, hidden_size=num_hiddens,bidirectional=True)
model = RNNModel(gru_layer, vocab_size).to(device)
train_and_predict_rnn_pytorch(model, num_hiddens, vocab_size, device,
                                corpus_indices, idx_to_char, char_to_idx,
                                num_epochs, num_steps, lr, clipping_theta,
                                batch_size, pred_period, pred_len, prefixes)

epoch 40, perplexity 1.000000, time 0.00 sec
 - 分开心心心心心心心心心心心心心心心心心心心心心心心心心心心心心心心心心心心心心心心心心心心心心心心心心心
 - 不分开心心心心心心心心心心心心心心心心心心心心心心心心心心心心心心心心心心心心心心心心心心心心心心心心心心
epoch 80, perplexity 1.000000, time 0.00 sec
 - 分开心心心心心心心心心心心心心心心心心心心心心心心心心心心心心心心心心心心心心心心心心心心心心心心心心心
 - 不分开心心心心心心心心心心心心心心心心心心心心心心心心心心心心心心心心心心心心心心心心心心心心心心心心心心
epoch 120, perplexity 1.000000, time 0.00 sec
 - 分开心心心心心心心心心心心心心心心心心心心心心心心心心心心心心心心心心心心心心心心心心心心心心心心心心心
 - 不分开心心心心心心心心心心心心心心心心心心心心心心心心心心心心心心心心心心心心心心心心心心心心心心心心心心
epoch 160, perplexity 1.000000, time 0.00 sec
 - 分开心心心心心心心心心心心心心心心心心心心心心心心心心心心心心心心心心心心心心心心心心心心心心心心心心心
 - 不分开心心心心心心心心心心心心心心心心心心心心心心心心心心心心心心心心心心心心心心心心心心心心心心心心心心
